In [ ]:
!pip install polars

In [ ]:
import polars as pl
import plotly.graph_objects as go
from google.colab import drive

# 1. Mount Drive (Nếu đã mount rồi thì dòng này sẽ báo drive already mounted, không sao cả)
drive.mount('/content/drive')

# Đổi đường dẫn file của bạn
file_path = '/content/drive/MyDrive/2019-Oct.csv'

print("Đang khởi tạo kế hoạch xử lý dữ liệu (Lazy Evaluation)...")

# 2. Xây dựng truy vấn (Query Plan)
q = (
    pl.scan_csv(file_path)
    .select(['event_time', 'event_type', 'brand', 'user_session'])
    .filter(pl.col('event_type') == 'view')

    # --- THAY ĐỔI QUAN TRỌNG: LỌC SẠCH DATA TRƯỚC ---
    # Bỏ các dòng không có brand (null) HOẶC brand là chuỗi 'unknown'
    .filter(pl.col('brand').is_not_null())
    .filter(pl.col('brand') != 'unknown')
    # -----------------------------------------------

    # Sắp xếp để tính luồng
    .sort(['user_session', 'event_time'])

    # Tính Next Click
    .with_columns(
        pl.col('brand').shift(-1).over('user_session').alias('next_brand')
    )

    # Loại bỏ dòng cuối của session (next_brand là null)
    .filter(pl.col('next_brand').is_not_null())

    # Loại bỏ click lặp lại liên tiếp (View Samsung -> View Samsung) để biểu đồ sạch hơn
    .filter(pl.col('brand') != pl.col('next_brand'))

    # Group by và đếm số lượng chuyển đổi
    .group_by(['brand', 'next_brand'])
    .len()
)

print("Đang thực thi và lọc bỏ 'unknown' (có thể mất 1-2 phút)...")
# 3. Thực thi (Execute)
df_flow = q.collect()

print("Hoàn tất xử lý!")
print(f"Tổng số cặp chuyển đổi (Brand vs Brand) tìm được: {len(df_flow)}")

# --- PHẦN VẼ BIỂU ĐỒ ---

# 1. Lấy Top 30 luồng lớn nhất
# Chuyển sang Pandas để vẽ
top_flows = df_flow.sort("len", descending=True).head(30).to_pandas()

# 2. Chuẩn bị Node và Index cho Sankey
all_nodes = list(set(top_flows['brand']).union(set(top_flows['next_brand'])))
node_map = {name: i for i, name in enumerate(all_nodes)}

source_indices = [node_map[src] for src in top_flows['brand']]
target_indices = [node_map[tgt] for tgt in top_flows['next_brand']]
values = top_flows['len'].tolist()

# 3. Vẽ Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color="skyblue"
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values,
        # Dùng màu xám nhạt trong suốt để dễ nhìn các đường chồng chéo
        color='rgba(169, 169, 169, 0.4)'
    )
)])

fig.update_layout(
    title_text="Top 30 Luồng Chuyển Đổi Thương Hiệu (Đã loại bỏ Unknown)",
    font_size=12,
    height=800
)

fig.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đang khởi tạo kế hoạch xử lý dữ liệu (Lazy Evaluation)...
Đang thực thi và lọc bỏ 'unknown' (có thể mất 1-2 phút)...
Hoàn tất xử lý!
Tổng số cặp chuyển đổi (Brand vs Brand) tìm được: 339387
